<a href="https://colab.research.google.com/github/YounSooKimTech/self_study/blob/main/NLP_ServiceRisk_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df_train = pd.read_excel("/content/BOS_RISKS (1).xlsx")
df_train.head()

df_train = df_train[df_train["Language"] == "English"]


In [2]:
df_train["Standard_risk_L2"].value_counts()

df_train['RISK_DESCRIPTION'].fillna("", inplace=True)
df_train["Standard_risk_L2"].fillna("", inplace=True)

df_train.head()

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,Language
2,Algeria,2023,Staff Turnover or change in Profile for key po...,Operational capacity,"2. Operational factors (funding, capacity and ...",Highly Possible,Significant Concern,Very High,Control,RC OFFICE ensure backup and follow up,English
4,Algeria,2023,Delays in implementing joint premises projects...,Agencies' commitment and participation,"1. Agencies alignment, commitment and particip...",Possible,Minor,Medium,Accept,monitor the case at UNCT level,English
5,Algeria,2023,Market service disruptions due to the impact o...,COVID-19,"3. External factors (COVID-19, economic, legis...",Possible,Slight Concern,Medium,Monitor,proactive action thru LTA and planning with Su...,English
7,Algeria,2023,Insufficient supplier capacity to handle incre...,Operational capacity,"2. Operational factors (funding, capacity and ...",Highly Possible,Slight Concern,High,Monitor,sensitize UNCT on capacity reinforcement,English
8,Angola,2023,Absence of common sharepoint would lead to ina...,Interoperability and data availability,"2. Operational factors (funding, capacity and ...",Negligible,Slight Concern,Medium,Control,Continue the maintenance of the Common Sharepo...,English


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # Import Logistic Regression
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier

# accuracy for logistic reg = 0.5, random forest = 0.99

# Load your labeled dataset into a DataFrame
labeled_df = df_train

# Preprocess the text data
X = labeled_df['RISK_DESCRIPTION']
y = labeled_df['Standard_risk_L2']

# Create a TF-IDF vectorizer to convert text to numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

# Transform the text data into TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Train a Logistic Regression model on the entire labeled dataset
# model = LogisticRegression()
model = RandomForestClassifier(n_estimators=100)
model.fit(X_tfidf, y)

# Predict labels for the entire labeled dataset
predicted_labels = model.predict(X_tfidf)

# Calculate accuracy by comparing predicted labels to actual labels
accuracy = (predicted_labels == y).mean()
print(f"Accuracy: {accuracy:.2f}")

# Add the predicted labels as a new column to your DataFrame
labeled_df['Predicted_Label'] = predicted_labels

labeled_df[["RISK_DESCRIPTION", "Standard_risk_L2", "Predicted_Label"]]

Accuracy: 0.99


,RISK_DESCRIPTION,Standard_risk_L2,Predicted_Label
2,Staff Turnover or change in Profile for key po...,Operational capacity,Operational capacity
4,Delays in implementing joint premises projects...,Agencies' commitment and participation,Agencies' commitment and participation
5,Market service disruptions due to the impact o...,COVID-19,COVID-19
7,Insufficient supplier capacity to handle incre...,Operational capacity,Operational capacity
8,Absence of common sharepoint would lead to ina...,Interoperability and data availability,Interoperability and data availability
...,...,...,...
502,Response to COVID and need to work from home m...,COVID-19,COVID-19
503,Response to COVID and need to work from home m...,COVID-19,COVID-19
504,Some agencies may not have flexible policies i...,Agencies' misalignment,Agencies' misalignment
505,Some agencies may not have flexible policies i...,Agencies' misalignment,Agencies' misalignment


In [4]:
labeled_df[labeled_df["Standard_risk_L2"] != labeled_df["Predicted_Label"]][["RISK_DESCRIPTION", "Standard_risk_L2", "Predicted_Label"]]

,RISK_DESCRIPTION,Standard_risk_L2,Predicted_Label
155,UN Organizations may not agree to utilize the ...,Agencies' misalignment,Agencies' commitment and participation
291,Not enough Agencies are willing to harmonize s...,Agencies' commitment and participation,Agencies' misalignment
332,The UN system will not be able to access the r...,Funding,Operational capacity
383,Different thresholds for audit and spot-checks...,Interoperability and data availability,Agencies' misalignment
427,Congruence of help desk needs and response sys...,Operational infrastructure,Interoperability and data availability


In [5]:
df_new = pd.read_excel("/content/SERVICES_RISKS.xlsx")

! pip install langdetect

from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'


df_new['Language'] = df_new["RISK_DESCRIPTION"].apply(detect_language)

df_new["Language"].value_counts()

en         802
es          63
fr          46
no          13
it           6
de           5
tl           3
da           2
af           2
cy           1
ro           1
ca           1
Unknown      1
Name: Language, dtype: int64

In [6]:
df_new["Language"].value_counts() / len(df_new)

en         0.847780
es         0.066596
fr         0.048626
no         0.013742
it         0.006342
de         0.005285
tl         0.003171
da         0.002114
af         0.002114
cy         0.001057
ro         0.001057
ca         0.001057
Unknown    0.001057
Name: Language, dtype: float64

In [7]:
unlabeled_df = df_new[df_new["Language"] == "en"]

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Load your labeled dataset into a DataFrame
labeled_df = df_train

# Preprocess the text data
X_train = labeled_df['RISK_DESCRIPTION']
y_train = labeled_df['Standard_risk_L2']

# Create a TF-IDF vectorizer to convert text to numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

# Transform the text data into TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Train a Random Forest Classifier model on the entire labeled dataset
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train_tfidf, y_train)

# Preprocess the text data in the unlabeled dataset
X_unlabeled = unlabeled_df['RISK_DESCRIPTION']
X_unlabeled_tfidf = tfidf_vectorizer.transform(X_unlabeled)

# Predict labels for the unlabeled dataset
predicted_labels = model.predict(X_unlabeled_tfidf)

# Add the predicted labels as a new column to the unlabeled DataFrame
unlabeled_df['Standard_risk_L2'] = predicted_labels

# Now, you have a new column 'Predicted_Label' in unlabeled_df with predicted labels.
# You can save this DataFrame to a file or use it as needed.


<ipython-input-8-30fcf5f607ce>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabeled_df['Standard_risk_L2'] = predicted_labels


In [9]:
unlabeled_df.head()

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,COMMON_SERVICE_LINE,COMMON_SERVICE_CATEGORY,COMMON_SERVICE,Language
0,Afghanistan,2023,A funding cut imposed by the legislative bodies,Funding,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en
1,Afghanistan,2023,A funding cut imposed by the legislative bodies,Funding,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en
2,Afghanistan,2023,A funding cut imposed by the legislative bodies.,Funding,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en
3,Afghanistan,2023,A funding cut imposed by the legislative bodies.,Funding,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Office space and accomodation,en
4,Afghanistan,2023,A funding cut imposed by the legislative bodies.,Funding,NaN,Possible,Significant Concern,High,Accept,Adjust service delivery according to the fundi...,Common Administration services and including c...,Transport Services,Vehicle maintenance,en


In [12]:
from google.colab import files

# Save the modified DataFrame with full language labels
unlabeled_df.to_excel('/content/eng_labeled.xlsx', index=False)

# Create a downloadable link for the saved CSV file
files.download('/content/eng_labeled.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>